In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import sqlite3
from transformers import AutoTokenizer, AutoModelForCausalLM

from scripts.llm import LLM, prepare_query, generate_sql

## Chargment du LLM pré-entraîné

In [3]:
model_id = "Qwen/Qwen2-1.5B-Instruct"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
    max_memory={0: "5GB"}
)

In [6]:
llm = LLM(model=model, tokenizer=tokenizer)

## Instanciation de la BDD

In [7]:
# creating file path
dbfile = "sqllite/atis-db.sqlite"
con = sqlite3.connect(dbfile)

## Requêtes SQL à tester:

1. `List ground transportation in BALTIMORE`

Résultat attendu:
| transport_type     |
| -----------------: |
| AIR TAXI OPERATION |
| LIMOUSINE          |
| RENTAL CAR         |

In [8]:
instruction = "List ground transportation in BALTIMORE"
query = prepare_query(instruction, "./prompt.json")
sql_generated = generate_sql(query, llm)
pd.read_sql_query(sql_generated, con)

,city_code,airport_code,transport_type,ground_fare


2. `what is the aircraft type with the greatest seating capacity`

Résultat attendu:
| aircraft_code |
| ------------: |
| 744           |

In [9]:
instruction = "what is the aircraft type with the greatest seating capacity" #"What's the aircraft type with the greatest seating capacity ?"
query = prepare_query(instruction, "./prompt.json")
sql_generated = generate_sql(query, llm)
pd.read_sql_query(sql_generated, con)

,aircraft_description
0,BOEING 747-400 PASSENGER


3. `What's the name of airport in Orlando ?`

Résultat attendu:
| airport_name          |
| ------------:         |
| ORLANDO INTERNATIONAL |

In [10]:
instruction = "What's the name of airport in Orlando ?"
query = prepare_query(instruction, "./prompt.json")
sql_generated = generate_sql(query, llm)
pd.read_sql_query(sql_generated, con)

,airport_name
0,ORLANDO INTERNATIONAL


4. `I need ground transportation in Dallas please show me what 's available`

Résultat attendu:
| transport_type |
| -------------: |
| LIMOUSINE      |
| RENTAL CAR     |

In [11]:
instruction = "I need ground transportation in Dallas please show me what 's available"
query = prepare_query(instruction, "./prompt.json")
sql_generated = generate_sql(query, llm)
pd.read_sql_query(sql_generated, con)

,city_code,airport_code,transport_type,ground_fare


5. `Show me the flights from Boston to Oakland`

Résultat attendu:
|flight_id |
| -------: |
| 302340   |
| 302341   |
| 302342   |
| 303432   |
| 304884   |
| 304885   |
| 306660   |
| 306661   |
| 306662   |
| 306663   |
| 308971   |
| 308972   |

In [12]:
instruction = "Tell me about flights from TORONTO to SALT LAKE CITY"
query = prepare_query(instruction, "./prompt.json")
sql_generated = generate_sql(query, llm)
pd.read_sql_query(sql_generated, con)

DatabaseError: Execution failed on sql ' SELECT * FROM flight WHERE departure_airport = 'TORONTO' AND TO_airport = 'SALT LAKE CITY';': no such column: departure_airport